# Парсинг:

In [71]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

In [72]:
#Параметры:

#Файл со ссылками"
output_file_list = r'z:/For Office/Ignatov/Python/parsing/links_list.xlsx'

#Файл с результатом парсинга:
output_file_parsing_report = r'z:/For Office/Ignatov/Python/parsing/parser_base.xlsx'

#Файл с байерским отчётом:
input_buyer_report = r'z:/For Office/Ignatov/Отчеты/01-Еженедельные/Байерский отчёт/ru.xlsx'

#Файл с листом сопоставлений:
input_mathing_list = r'z:/For Office/Ignatov/Python/parsing/matching_list.xlsx'

#Файл итоговый объединенный отчёт:
output_file_parsing_final = r'z:/For Office/Ignatov/Python/parsing/parsing_final.xlsx'

#Кол-во страниц для парсинга
parsing_pages_count = 2000

In [73]:
list_pages = []
for i in tqdm(range(1 , parsing_pages_count)):
    URL = 'https://www.detmir.ru/search/results/order/price-desc/page/'+str(i)
    req = requests.get(URL)
    soup_page = BeautifulSoup(req.text, 'lxml')
    for row in soup_page.find_all('a', attrs = {'class':'KS kD'}):
        list_pages.append('https://www.detmir.ru'+row.attrs["href"])
print(len(list_pages))

100%|██████████████████████████████████████████████████████████████████████████████| 1999/1999 [58:43<00:00,  1.76s/it]

53081


In [74]:
#Сохраняем список ссылок в exel:
links_list = pd.DataFrame(list_pages)
links_list.to_excel(output_file_list, index=False)

In [75]:
#Загружаем список ссылок обратно в p:
list_pages = pd.read_excel(output_file_list, header=0)
#Преобразуем в список:
list_pages.columns = ['URL']
list_pages = list_pages['URL'].tolist()

In [76]:
table = pd.DataFrame()

for element in tqdm(list_pages):
    req = requests.get(element)
    soup_page = BeautifulSoup(req.text, 'lxml') 
    #Сбор иерархии
    #hierarchy_lvl_1
    try:
        hierarchy_lvl_1 = soup_page.find_all('a', attrs = {'class':'II'})[1].text
    except:
        hierarchy_lvl_1 = float('NaN')
    #hierarchy_lvl_2
    try:
        hierarchy_lvl_2 = soup_page.find_all('a', attrs = {'class':'II'})[2].text
    except:
        hierarchy_lvl_2 = float('NaN')    
    #hierarchy_lvl_3
    try:
        hierarchy_lvl_3 = soup_page.find_all('a', attrs = {'class':'II'})[3].text
    except:
        hierarchy_lvl_3 = float('NaN')    
    #hierarchy_lvl_4
    try:
        hierarchy_lvl_4 = soup_page.find_all('a', attrs = {'class':'II'})[4].text
    except:
        hierarchy_lvl_4 = float('NaN')   
    # Наименование товара
    try:
        name_item = soup_page.find('h1', attrs = {'class':'Gv'}).text
    except:
        name_item = float('NaN')
    # Бренд
    try:    
        brand = soup_page.find('p', attrs = {'class':'Pk'}).text
    except:
        brand = float('NaN')
    #item включает номер артикула и код товара:
    try:
        item = soup_page.find('ul', attrs = {'class':'eJ rQ rq eK'}).text
    except:
        item = float('NaN')
    #item_art включает номер артикула:
    try:
        item_art = (item.split('Код товара')[0]).partition('Артикул ')[2]
    except:
        item_art = float('NaN')
    #item_good включает код товара:
    try:
        item_goods = item.partition('Код товара ')[2]
    except:
        item_goods = float('NaN')
    # Текущая цена
    try:   
        current_price = soup_page.find('div', attrs = {'class':'qa'}).text
    except:
        current_price = float('NaN')
    # Прошлая цена
    try:
        previous_price = soup_page.find('span', attrs = {'class':'qb'}).text
    except:
        previous_price = current_price
    # Кол-во отзывов   
    try:
        feedbacks = soup_page.find('span', attrs = {'class':'rl'}).text
    except:
        feedbacks = float('NaN')
    #считываем кол-во строк с текстом '90oSMiN', это соответвтует кол-ву 0,5 звёзд
    try:
        rating = int(str(soup_page.find_all('div', attrs = {'class':'Ls LV rR'})).count('Lt LU')) / 2
    except:
        rating = float('NaN')
    new_row = pd.Series([element, hierarchy_lvl_1, hierarchy_lvl_2, hierarchy_lvl_3, hierarchy_lvl_4,
                     name_item, brand, item_art, item_goods, current_price, previous_price, feedbacks, rating])    
    table = table.append(new_row, ignore_index=True)    

100%|██████████████████████████████████████████████████████████████████████████| 53081/53081 [5:19:34<00:00,  2.77it/s]


In [77]:
table.columns = ['link', 'hierarchy_lvl_1', 'hierarchy_lvl_2', 'hierarchy_lvl_3', 'hierarchy_lvl_4',
                     'name_item', 'brand', 'item_art', 'item_goods', 'current_price', 'previous_price', 'feedbacks', 'rating']

In [78]:
table.head(10)

,link,hierarchy_lvl_1,hierarchy_lvl_2,hierarchy_lvl_3,hierarchy_lvl_4,name_item,brand,item_art,item_goods,current_price,previous_price,feedbacks,rating
0,https://www.detmir.ru/product/index/id/1000053...,Прогулки и путешествия,Детские коляски,Детские коляски 2 в 1,NaN,Коляска Bugaboo Fox 2в1 Grey Melange base Black,Bugaboo,,1000053655nbr,99 044 ₽,99 044 ₽,Оставить отзыв,0.0
1,https://www.detmir.ru/product/index/id/3137610/,Прогулки и путешествия,Детские коляски,Детские коляски 2 в 1,NaN,Коляска 2в1 Bugaboo Fox Classic Complete Black...,Bugaboo,239171AE01,1000072639,97 289 ₽,97 289 ₽,Оставить отзыв,0.0
2,https://www.detmir.ru/product/index/id/3137621/,Прогулки и путешествия,Детские коляски,Детские коляски 2 в 1,NaN,Коляска 2в1 Bugaboo Fox Classic Complete Black...,Bugaboo,239171AI01,1000072650,97 289 ₽,97 289 ₽,Оставить отзыв,0.0
3,https://www.detmir.ru/product/index/id/1000053...,Прогулки и путешествия,Детские коляски,Детские коляски 2 в 1,NaN,Коляска Bugaboo Fox 2в1 GREY MELANGE Base Alu,Bugaboo,,1000053653nbr,94 244 ₽,94 244 ₽,Оставить отзыв,0.0
4,https://www.detmir.ru/product/index/id/3019510/,Прогулки и путешествия,Детские коляски,Детские коляски 2 в 1,NaN,Коляска 2 в 1 Bugaboo Mono Donkey 2 Complete G...,Bugaboo,189150GR01,1000050636,92 989 ₽,92 989 ₽,Оставить отзыв,0.0
5,https://www.detmir.ru/product/index/id/3137609/,Прогулки и путешествия,Детские коляски,Детские коляски 2 в 1,NaN,Коляска 2в1 Bugaboo Fox Classic Complete Alu/d...,Bugaboo,239170AA01,1000072638,92 689 ₽,92 689 ₽,Оставить отзыв,0.0
6,https://www.detmir.ru/product/index/id/1000053...,Прогулки и путешествия,Детские коляски,Детские коляски 2 в 1,NaN,Коляска Bugaboo Fox 2в1 Black/SKY BLUE base Alu,Bugaboo,,1000053651nbr,92 134 ₽,92 134 ₽,Оставить отзыв,0.0
7,https://www.detmir.ru/product/index/id/3085787/,Прогулки и путешествия,Детские коляски,Детские коляски 2 в 1,NaN,Коляска 2в1 Hartan Vip GTX XL 710 без сумки Cе...,Hartan,2289.106.710,1000054723,87 489 ₽,87 489 ₽,Оставить отзыв,0.0
8,https://www.detmir.ru/product/index/id/3085789/,Прогулки и путешествия,Детские коляски,Детские коляски 2 в 1,NaN,Коляска 2в1 Hartan Racer GTS XL 716 Cеро-вишне...,Hartan,2590.042.716,1000054725,87 489 ₽,87 489 ₽,1 отзыв,4.0
9,https://www.detmir.ru/product/index/id/1000079...,Прогулки и путешествия,Детские коляски,Детские коляски 2 в 1,NaN,Коляска Cybex 2в1 Priam Light Seat Happy Black...,Cybex,,1000079137nbr,78 878 ₽,78 878 ₽,Оставить отзыв,0.0


In [79]:
table.to_excel(output_file_parsing_report, index=False)

# Обработка таблицы:

In [80]:
table = pd.read_excel(output_file_parsing_report, header=0)

In [81]:
#Приводи цены к правильному формату
def price_correction1(data):
    try:
        data['current_price'] = data['current_price'].split(' ₽')[0].replace(' ', '')
        return data['current_price'].replace(',', '.')
    except:
        return 0
table['current_price'] = table.apply(price_correction1, axis=1)

def price_correction2(data):
    try:
        data['previous_price'] = data['previous_price'].split(' ₽')[0].replace(' ', '')
        return data['previous_price'].replace(',', '.')
    except:
        return 0
table['previous_price'] = table.apply(price_correction2, axis=1)

In [82]:
#Превращаем отзывы в число:
import re
def feedbacks_correction(data):
    try:
        f_count = re.findall('\d+', data['feedbacks'])
        if f_count:
            return f_count[0]
        else:
            return 0
    except:
        return float('NaN')
table['feedbacks'] = table.apply(feedbacks_correction, axis=1)

In [83]:
table['current_price'] = table['current_price'].astype('float64')
table['previous_price'] = table['previous_price'].astype('float64')
table['feedbacks'] = table['feedbacks'].astype('float64')

In [84]:
#Удаляем строки с нулевыми ценами:
table = table[table['current_price']>0]
#Считаем % скидки:
table['discount'] = table['current_price'] / table['previous_price'] - 1

In [85]:
#Сохраняем результат
table.to_excel(output_file_parsing_report, index=False)

# Объединение таблиц

In [98]:
#Загружаем результат парсинга
parsing_report = pd.read_excel(output_file_parsing_report, header=0)
#Заполняем столбец с ратикулом "-", чтобы не было косяков при объединении
parsing_report['item_art'] = parsing_report['item_art'].fillna('-')

In [99]:
bayer_report = pd.read_excel(input_buyer_report, sheet_name='Номенклатуры', header=1)

In [100]:
#Получаем из байерского отчёта только таблицу с нужными столбцами:
bayer_report = bayer_report[[
    'Артикул',
    'Наименование',
    'Байер',
    'Группа',
    'Прайс-поинт на начало продаж, нац. вал.',
    'Прайс-поинт текущий, нац. вал.',
    'Остаток на текущий понедельник',
    'Продажи за неделю, шт.',
    'Выручка нац.вал. 7 дней, с НДС',
    'Маржа % за 7 дней нац.вал.',
    'Модель',
    'Гиперссылка',
    'Поставщик',
    'Торговая марка'
]]
#Меняем имя столбца с артикулом поставщика, чтобы объединить таблицы:
bayer_report['final_key_kk'] = bayer_report['Модель'] + bayer_report['Артикул']

In [101]:
#Загружаем список ручных сопоставлений-ключей
mathing_list = pd.read_excel(input_mathing_list, header=1)

In [102]:
#Создаем ключ в таблице ДМ
parsing_report['item_art'] = parsing_report['item_art'].astype(str)
parsing_report['item_goods'] = parsing_report['item_goods'].astype(str)
parsing_report['final_key_dm'] = parsing_report['item_art'] + parsing_report['item_goods']

In [103]:
#Подкачиваем ключи в таблицу:
parsing_report = parsing_report.merge(mathing_list[['final_key_dm', 'final_key_kk']], on='final_key_dm', how='left')

In [104]:
#Пустые заполним прочерком
parsing_report['final_key_kk'] = parsing_report['final_key_kk'].fillna('-')

In [105]:
#Объединяем
table_joint = parsing_report.merge(bayer_report, on='final_key_kk', how='left')

In [106]:
#Добавляем среднюю цену продажи за 7 дней КК:
table_joint['Средняя цена продажи 7 дней, нац.вал с НДС'] = table_joint[
    'Выручка нац.вал. 7 дней, с НДС'] / table_joint['Продажи за неделю, шт.']
table_joint['Разница тек. цен КК/ДМ'] = table_joint['Прайс-поинт текущий, нац. вал.'] / table_joint['current_price'] - 1

In [107]:
#Переименовываем столбцы
table_joint.columns = [
    ### Столбцы Детский Мир:
    'Ссылка',
    'Иерархия №1',
    'Иерархия №2',
    'Иерархия №3',
    'Иерархия №4',
    'Наименование товара',
    'Бренд',
    'Артикул поставщика',
    'Артикул внутренний',
    'Текущая цена',
    'Первоначальная цена',
    'Кол-во отзывов',
    'Рейтинг',
    'Скидка, %',
    'final_key_dm',
    'final_key_kk',
    ### Столбцы КариКидс:
    'Артикул КК',
    'Наименование артикула',
    'Байер',
    'Группа',
    'Прайс-поинт на начало продаж, нац. вал.',
    'Прайс-поинт текущий, нац. вал.',
    'Остаток на текущий понедельник',
    'Продажи за неделю, шт.',
    'Выручка нац.вал. 7 дней, с НДС',
    'Маржа % за 7 дней нац.вал.',
    'Модель поставщика КК',
    'Ссылка КК',
    'Поставщик',
    'Торговая марка',
    'Средняя цена продажи 7 дней, нац.вал с НДС',
    'Разница тек. цен КК/ДМ'
]

In [108]:
#Меняем порядок столбцов
table_joint = table_joint[[
    #Артикула ДМ
    'Ссылка',
    'Иерархия №1',
    'Иерархия №2',
    'Иерархия №3',
    'Иерархия №4',
    'Наименование товара',
    'Бренд',
    'Артикул поставщика',
    'Артикул внутренний',
    'Первоначальная цена',
    'Текущая цена',
    'Скидка, %',
    'Кол-во отзывов',
    'Рейтинг',
    'Разница тек. цен КК/ДМ',
    #Артикула КК
    'Модель поставщика КК',
    'Ссылка КК',
    'Артикул КК',
    'Наименование артикула',
    'Поставщик',
    'Торговая марка',
    'Байер',
    'Группа',
    'Прайс-поинт на начало продаж, нац. вал.',
    'Прайс-поинт текущий, нац. вал.',
    'Средняя цена продажи 7 дней, нац.вал с НДС',
    'Остаток на текущий понедельник',
    'Продажи за неделю, шт.',
    'Маржа % за 7 дней нац.вал.',
]]

# Выгрузка в exel

In [109]:
table_joint.to_excel(output_file_parsing_final, index=False)